In [0]:
spark
sc = spark.sparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql import functions as F
import seaborn as sns
import numpy as np
from pyspark.sql.types import IntegerType
import matplotlib.pyplot as plt

In [0]:
filepath=('s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json')

steam_log = (spark.read.format('json')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load(filepath))

In [0]:
steam_log.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
# Assuming steam_log is your DataFrame
check_data = steam_log \
    .withColumn('categories', F.col('data').getField('ccudd')) \

# Limitez le nombre de lignes affichées et montrez le DataFrame
check_data.limit(10).show()

In [0]:
type(steam_log)
steam_log.columns

['data', 'id']

Macro-Level Analysis

In [0]:
#Which publisher has released the most games on Steam?

publisher_data = steam_log\
                .withColumn('publisher', F.col('data').getField('publisher'))

publisher_data.createOrReplaceTempView('publisher_data')
ranking_publisher = spark.sql("""Select publisher, count(publisher) as nb_games 
                                from publisher_data 
                                group by publisher 
                                order by nb_games desc""").show(5)




+--------------+--------+
|     publisher|nb_games|
+--------------+--------+
|Big Fish Games|     422|
|        8floor|     202|
|          SEGA|     165|
|Strategy First|     151|
|   Square Enix|     141|
+--------------+--------+
only showing top 5 rows



In [0]:
#What are the best rated games?

game_data = steam_log\
            .withColumn('name', F.col('data').getField('name'))\
            .withColumn('positive', F.col('data').getField('positive'))\
            .drop('data','id')

ranking_games = game_data.orderBy(F.col('positive').desc(), 'name').show(5)


+--------------------+--------+
|                name|positive|
+--------------------+--------+
|Counter-Strike: G...| 5943345|
|              Dota 2| 1534895|
|  Grand Theft Auto V| 1229265|
| PUBG: BATTLEGROUNDS| 1185361|
|            Terraria| 1014711|
+--------------------+--------+
only showing top 5 rows



In [0]:
#Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

yearly_analysis_data = steam_log\
                    .withColumn('year', F.col('data').getField('release_date'))\
                    .withColumn('year', F.split(F.col('year'), '/').getItem(0)) \
                    .withColumn('name', F.col('data').getField('name'))\
                    .drop('data','id')

yearly_analysis_data.createOrReplaceTempView('yearly_analysis_data')
ranking_years=spark.sql('Select year, count(year) as nb_games \
                        from yearly_analysis_data \
                        group by year \
                        order by nb_games desc').display()

year,nb_games
2021,8823
2020,8305
2018,7678
2022,7455
2019,6968
2017,6017
2016,4185
2015,2576
2014,1557
2013,471


Databricks visualization. Run in Databricks to view.

In [0]:
#How are the prizes distributed?
prices_data = steam_log\
              .withColumn('price', F.col('data').getField('price'))\
              .drop('data','id')


"""#price est une chaine de caractère et bloque la suite donc transformations en integer
prices_data = prices_data.withColumn('price_int', prices_data.price.cast(IntegerType())).drop('price')

#pyspark ne gère pas les plots donc transformation en df pandas pour utiliser seaborn pour la représentation à proprement parler et matplot lib pour les explication en side
prices_data_pd = prices_data.toPandas()

#on passe en log sinon la visualisation est illisible - trop petit
prices_data_pd['log_price_int'] = np.log(prices_data_pd['price_int'])

plt.figure(figsize=(10, 6))
sns.histplot(prices_data_pd['log_price_int'], bins=30, kde=True)
plt.xlabel('Log Price')
plt.ylabel('Frequency')
plt.title('Log Price Distribution')
plt.show()"""

price
999
999
599
1999
199
799
1299
0
299
1399


"#price est une chaine de caractère et bloque la suite donc transformations en integer\nprices_data = prices_data.withColumn('price_int', prices_data.price.cast(IntegerType())).drop('price')\n\n#pyspark ne gère pas les plots donc transformation en df pandas pour utiliser seaborn pour la représentation à proprement parler et matplot lib pour les explication en side\nprices_data_pd = prices_data.toPandas()\n\n#on passe en log sinon la visualisation est illisible - trop petit\nprices_data_pd['log_price_int'] = np.log(prices_data_pd['price_int'])\n\nplt.figure(figsize=(10, 6))\nsns.histplot(prices_data_pd['log_price_int'], bins=30, kde=True)\nplt.xlabel('Log Price')\nplt.ylabel('Frequency')\nplt.title('Log Price Distribution')\nplt.show()"

In [0]:
#Are there many games with a discount?

discount_data = steam_log\
                .withColumn('name',F.col('data').getField('name'))\
                .withColumn('discount',F.col('data').getField('discount'))\
                .drop('data','id')


discount_data.filter(discount_data.discount > 0).distinct().count()


2518

In [0]:
#What are the most represented languages?

language_data = steam_log\
                .withColumn('languages',F.col('data').getField('languages'))\
                .drop('data','id')
                
# Split the languages column by commas
language_data = language_data.withColumn('languages', F.split(F.col('languages'), ','))
# Explode the split column to create a new row for each language
language_data = language_data.withColumn('language_unique', F.explode(F.col('languages')))
language_data = language_data.drop('languages')

language_counts = language_data.groupBy('language_unique').count().orderBy('count', ascending=False)
display(language_counts.limit(5))


In [0]:
#Are there many games prohibited for children under 16/18?

required_age_data = steam_log\
                .withColumn('required_age',F.col('data').getField('required_age'))\
                .drop('data','id')

required_age_data_analysis = required_age_data.groupby('required_age').count().orderBy('count', ascending=False).show()


Genre Analysis

In [0]:
#What are the most represented genres?

genre_data = steam_log\
                .withColumn('genre',F.col('data').getField('genre'))\
                .drop('data','id')

genre_data.createOrReplaceTempView('genre_data')
ranking_genre = spark.sql("""Select genre, count(genre) as nb_genre 
                                from genre_data 
                                group by genre 
                                order by nb_genre desc""").limit(5).display()

genre,nb_genre
"Action, Indie",3460
"Casual, Indie",3060
"Action, Adventure, Indie",2783
"Adventure, Indie",2316
"Action, Casual, Indie",1914


In [0]:
#Are there any genres that have a better positive/negative review ratio?

genre_ration_data = steam_log\
                .withColumn('genre',F.col('data').getField('genre'))\
                .withColumn('positive',F.col('data').getField('positive'))\
                .withColumn('negative',F.col('data').getField('negative'))\
                .withColumn('ratio', 100*F.col('positive') / (F.col('negative')+F.col('positive')))\
                .orderBy('ratio', ascending=False)\
                .drop('data','id').limit(10).display()


genre,positive,negative,ratio
"Casual, Simulation, Strategy",17,0,100.0
"Action, Adventure, Casual, Indie",12,0,100.0
"Action, Indie",4,0,100.0
Indie,2,0,100.0
"Action, Adventure, Indie",7,0,100.0
"Action, Casual, Indie",9,0,100.0
"Adventure, Indie, RPG",1,0,100.0
"Casual, Indie, Racing, Sports",1,0,100.0
"Action, Adventure, Casual, RPG",11,0,100.0
"Casual, Indie",6,0,100.0


In [0]:
#Do some publishers have favorite genres?

# Assuming steam_log is your DataFrame
publisher_genre_data = steam_log \
    .withColumn('publisher', F.col('data').getField('publisher')) \
    .withColumn('genre', F.col('data').getField('genre'))

# Si 'genre' est une chaîne de caractères séparée par des virgules, transformez-la en un tableau
publisher_genre_data = publisher_genre_data \
    .withColumn('genre', F.split(F.col('genre'), ','))

# Utilisez 'explode' pour transformer chaque élément du tableau en une ligne séparée
publisher_genre_data = publisher_genre_data \
    .withColumn('genre_explode', F.explode('genre')) \
    .drop('data', 'id', 'genre')

# Affichez les 5 premières lignes du DataFrame résultant
publisher_genre_data.groupBy('publisher', 'genre_explode').count()\
                    .orderBy('count', ascending=False)\
                    .limit(5).display()

publisher,genre_explode,count
Big Fish Games,Adventure,391
Big Fish Games,Casual,389
8floor,Casual,194
Choice of Games,RPG,136
Choice of Games,Indie,112


In [0]:
#What are the most lucrative genres?

ccu_data = steam_log \
    .withColumn('genre', F.col('data').getField('genre')) \
    .withColumn('ccu', F.col('data').getField('ccu')) \
    .groupBy('genre') \
    .agg(F.sum('ccu').alias('ccu')) \
    .drop('data', 'id') \
    .limit(10)

# Afficher le DataFrame résultant
ccu_data.orderBy('ccu',ascending=False).show()


+--------------------+----+
|               genre| ccu|
+--------------------+----+
|Animation & Model...|4877|
|Adventure, Casual...|4677|
|Casual, Indie, Si...|1383|
|Adventure, Free t...| 516|
|Action, Simulatio...| 493|
|Casual, Simulatio...| 131|
|Audio Production,...| 110|
|Action, Casual, F...|   2|
|Action, Adventure...|   0|
|Action, Adventure...|   0|
+--------------------+----+



Platform analysis

In [0]:
#Are most games available on Windows/Mac/Linux instead? (1)

availabilty_data = steam_log \
    .withColumn('name', F.col('data').getField('name')) \
    .withColumn('platforms', F.col('data').getField('platforms')) \
    .withColumn('linux', F.when(F.col('platforms').getField('linux'), 1).otherwise(0)) \
    .withColumn('mac', F.when(F.col('platforms').getField('mac'), 1).otherwise(0)) \
    .withColumn('windows', F.when(F.col('platforms').getField('windows'), 1).otherwise(0)) \
    .withColumn('total_platforms', F.col('linux') + F.col('mac') + F.col('windows')) \
    .drop('data', 'id', 'platforms')

availabilty_data.createOrReplaceTempView('availabilty_data')
availabilty_data_sql = spark.sql("""Select sum(linux), sum(mac), sum(windows)
                                from availabilty_data 
                                """).display()

sum(linux),sum(mac),sum(windows)
8458,12770,55676


In [0]:
#Are most games available on Windows/Mac/Linux instead? (2)

availabilty_data = steam_log \
    .withColumn('name', F.col('data').getField('name')) \
    .withColumn('platforms', F.col('data').getField('platforms')) \
    .withColumn('linux', F.when(F.col('platforms').getField('linux'), 1).otherwise(0)) \
    .withColumn('mac', F.when(F.col('platforms').getField('mac'), 1).otherwise(0)) \
    .withColumn('windows', F.when(F.col('platforms').getField('windows'), 1).otherwise(0)) \
    .withColumn('total_platforms', F.col('linux') + F.col('mac') + F.col('windows')) \
    .drop('data', 'id', 'platforms') \
    

# Afficher le DataFrame résultant
availabilty_data.groupBy('total_platforms').count().orderBy('total_platforms', ascending=True).show()

+---------------+-----+
|total_platforms|count|
+---------------+-----+
|              1|41285|
|              2| 7599|
|              3| 6807|
+---------------+-----+



In [0]:
#Do certain genres tend to be preferentially available on certain platforms?

availabilty_genre_data = steam_log \
    .withColumn('genre', F.col('data').getField('genre')) \
    .withColumn('genre', F.split(F.col('genre'), ','))\
    .withColumn('genre_explode', F.explode('genre')) \
    .withColumn('platforms', F.col('data').getField('platforms')) \
    .withColumn('linux', F.when(F.col('platforms').getField('linux'), 1).otherwise(0)) \
    .withColumn('mac', F.when(F.col('platforms').getField('mac'), 1).otherwise(0)) \
    .withColumn('windows', F.when(F.col('platforms').getField('windows'), 1).otherwise(0)) \
    .withColumn('total_platforms', F.col('linux') + F.col('mac') + F.col('windows')) \
    .drop('data', 'id', 'platforms','genre')


# Regrouper par 'genre_explode' et calculer la somme des valeurs des colonnes 'linux', 'mac', et 'windows'
platform_sums = availabilty_genre_data \
    .groupBy('genre_explode') \
    .agg(
        F.sum('linux').alias('sum_linux'),
        F.sum('mac').alias('sum_mac'),
        F.sum('windows').alias('sum_windows'),
        F.sum('total_platforms').alias('total_platforms')
    ) \
    .orderBy('total_platforms', ascending=False)

# Afficher le DataFrame résultant
platform_sums.show()


+--------------------+---------+-------+-----------+---------------+
|       genre_explode|sum_linux|sum_mac|sum_windows|total_platforms|
+--------------------+---------+-------+-----------+---------------+
|               Indie|     5912|   8367|      34267|          48546|
|              Action|     3369|   4549|      23612|          31530|
|              Casual|     1838|   2776|      12309|          16923|
|           Adventure|     1945|   3150|      11262|          16357|
|            Strategy|     1707|   2769|      10042|          14518|
|              Casual|     1467|   2354|       9773|          13594|
|           Adventure|     1357|   1889|      10165|          13411|
|          Simulation|     1399|   2191|       9626|          13216|
|                 RPG|     1438|   2117|       8718|          12273|
|               Indie|     1066|   1568|       5409|           8043|
|        Early Access|      630|    896|       6125|           7651|
|        Free to Play|      381|  